In [1]:
### Run this cell before continuing. 
library(tidyverse)
library(scales)
library(repr)
library(stringr)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘scales’


The following object is masked from ‘package:purrr’:

    discard


The following object is masked from ‘package:readr’:

    col_factor




In [2]:
training_data <- read_csv("data/Training_Data.csv")
training_data

Parsed with column specification:
cols(
  STG = col_double(),
  SCG = col_double(),
  STR = col_double(),
  LPR = col_double(),
  PEG = col_double(),
  UNS = col_character()
)



STG,SCG,STR,LPR,PEG,UNS
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
0.00,0.00,0.00,0.00,0.00,very_low
0.08,0.08,0.10,0.24,0.90,High
0.06,0.06,0.05,0.25,0.33,Low
⋮,⋮,⋮,⋮,⋮,⋮
0.54,0.82,0.71,0.29,0.77,High
0.50,0.75,0.81,0.61,0.26,Middle
0.66,0.90,0.76,0.87,0.74,High


In [3]:
data_summary <- training_data %>%
                group_by(UNS) %>%
                summarize(n = n()) %>%
                arrange(desc(n))

`summarise()` ungrouping output (override with `.groups` argument)



In [4]:
data_summary_wider <- pivot_wider(data_summary,  names_from = UNS,  values_from = n)

In [5]:
training_data_summary_stats <- summarize(training_data,  mean_STG = mean(STG), mean_SCG = mean(SCG),  mean_STR = mean(STR), mean_LPR = mean(LPR), mean_PEG = mean(PEG)) 

In [6]:
final_summary <- merge(training_data_summary_stats, data_summary_wider)

final_summary

mean_STG,mean_SCG,mean_STR,mean_LPR,mean_PEG,Middle,Low,High,very_low
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>
0.3711473,0.3556744,0.4680039,0.4327132,0.4585388,88,83,63,24
